<a href="https://colab.research.google.com/github/AnaJuliaMM/final_chatbot/blob/main/final_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependências Externas

In [161]:
import itertools
import nltk
import pandas as pd
import string

from dataclasses import dataclass, field
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import mac_morpho
from nltk.tag import UnigramTagger
from nltk import pos_tag
from nltk.stem import RSLPStemmer
from typing import List, Optional, Callable


In [ ]:
nltk.download('punkt')
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('mac_morpho')

# Dados para consulta

In [134]:
tasks = {
    'task_id': [1, 2, 3, 4, 5],
    'task_name': [
        'Comprar leite',
        'Enviar e-mail',
        'Reunião com a equipe',
        'Revisar relatório',
        'Organizar arquivos'
    ],
    'status': ['pendente', 'concluída', 'pendente', 'pendente', 'concluída'],
    'created_at': pd.to_datetime([
        '2023-10-01 09:00:00',
        '2023-10-01 10:00:00',
        '2023-10-02 11:00:00',
        '2023-10-03 12:30:00',
        '2023-10-04 14:15:00'
    ]),
    'due_date': pd.to_datetime([
        '2023-10-05',
        '2023-10-02',
        '2023-10-04',
        '2023-10-05',
        '2023-10-06'
    ])
}

tasks_df = pd.DataFrame(tasks)

In [135]:
tasks_df

,task_id,task_name,status,created_at,due_date
0,1,Comprar leite,pendente,2023-10-01 09:00:00,2023-10-05
1,2,Enviar e-mail,concluída,2023-10-01 10:00:00,2023-10-02
2,3,Reunião com a equipe,pendente,2023-10-02 11:00:00,2023-10-04
3,4,Revisar relatório,pendente,2023-10-03 12:30:00,2023-10-05
4,5,Organizar arquivos,concluída,2023-10-04 14:15:00,2023-10-06


# Funções de manipulação dos dados

In [109]:
def add_task():
    print("Tarefa adicionada com sucesso")

In [148]:
def list_tasks():
    print("Tarefas no sistema")
    return tasks_df

In [149]:
def find_task():
    print("Tarefa encontrada")



In [138]:
def complete_task():
    pass

# Estrutura da Árvore de Decisão

**Explicação da árvore:**

bla bla bla bla

**Níveis**

0.  Raiz
1. Entidades (substantivos)
2. Intenções (verbo)
3. Funções (funções Python que vão executar operações no dataframe com dados)



In [108]:
@dataclass
class Node:
    """Representa um nó da árvore com um valor e filhos."""
    value: str
    children: List['Node'] = field(default_factory=list)

class Tree:
    """Classe que representa uma árvore, onde cada nó pode ter múltiplos filhos.
    Permite a adição de nós, busca de valores e impressão hierárquica.
    """

    def __init__(self, value: str, stemmer=None):
        """Inicializa a árvore com um nó raiz e um stemmer opcional.

        Args:
            value (str): Valor do nó raiz.
            stemmer: Função de stemming opcional para normalizar valores.
        """
        self.root = Node(value)
        self.stemmer = stemmer

    def add_child(self, parent_value: str, value: str, stem: bool = False):
        """Adiciona um nó filho a um nó pai especificado.

        Args:
            parent_value (str): Valor do nó pai onde o filho será adicionado.
            value (str): Valor do novo nó filho.
            stem (bool): Se True, aplica stemming ao valor do nó filho.

        Raises:
            ValueError: Se o nó pai não for encontrado.
        """
        if stem and self.stemmer:
            value = self.stemmer.stem(value)

        parent_node = self.search(parent_value)
        if not parent_node:
            raise ValueError(f"Nó pai '{parent_value}' não encontrado.")

        parent_node.children.append(Node(value))

    def search(self, value: str, node: Optional[Node] = None) -> Optional[Node]:
        """Busca por um nó com um valor específico na árvore.

        Args:
            value (str): Valor a ser buscado.
            node (Node, opcional): Nó inicial para a busca. Se não informado, começa pela raiz.

        Returns:
            Node ou None: Retorna o nó encontrado ou None se não encontrar.
        """
        if node is None:
            node = self.root

        if self._match_value(node.value, value):
            return node

        for child in node.children:
            result = self.search(value, child)
            if result:
                return result
        return None

    def print(self, node: Optional[Node] = None, level: int = 0):
        """Imprime a árvore em um formato hierárquico.

        Args:
            node (Node, opcional): Nó inicial para impressão. Se não informado, começa pela raiz.
            level (int): Nível atual na hierarquia, usado para indentação.
        """
        if node is None:
            node = self.root

        print("  " * level + str(node.value))
        for child in node.children:
            self.print(child, level + 1)

    def find_answer(self, entity: str, intent: str) -> Optional[str]:
        """Encontra a resposta associada a uma entidade e um intento.

        Args:
            entity (str): Entidade a ser buscada.
            intent (str): Intento relacionado à entidade.

        Returns:
            str ou None: A resposta associada, ou None se não encontrar.
        """
        entity_node = self.search(self._stem_if_needed(entity))
        if entity_node:
            intent_node = self.search(self._stem_if_needed(intent), entity_node)
            if intent_node and intent_node.children:
                return intent_node.children[0].value
        return None

    def _stem_if_needed(self, value: str) -> str:
        """Aplica o stemming ao valor se o stemmer estiver configurado.

        Args:
            value (str): Valor a ser processado.

        Returns:
            str: Valor processado com stemming, se aplicável.
        """
        return self.stemmer.stem(value) if self.stemmer else value

    def _match_value(self, node_value: str, search_value: str) -> bool:
        """Verifica se o valor do nó corresponde ao valor buscado, considerando o stemming.

        Args:
            node_value (str): Valor do nó atual.
            search_value (str): Valor buscado.

        Returns:
            bool: True se os valores coincidem, False caso contrário.
        """
        search_value = self._stem_if_needed(search_value)
        return node_value == search_value


# Preenchimento da Árvore de Decisão

In [150]:
# Intanciar objeto para stemmizar
stemmer = RSLPStemmer()
# Instanciar Árvore de decisão
tree = Tree("chatbot")

⚠️⚠️⚠️**A fazer:**
Ao preencher a árvore, ao invés de colocar uma string, colocar lista de sinônimos stemmerizados



In [151]:
# Entidades
tree.add_child("chatbot", "tarefa")

# Intenções
tree.add_child("tarefa", "adicionar")
tree.add_child("tarefa", "encontrar")
tree.add_child("tarefa", "listar")
tree.add_child("tarefa", "concluir")

# Respostas
tree.add_child("adicionar", add_task)
tree.add_child("listar", list_tasks)

In [141]:
tree.print()

chatbot
  tarefa
    adicionar
      <function add_task at 0x7f2484c1a200>
    encontrar
    listar
      <function list_tasks at 0x7f248326cdc0>
    concluir


Teste de chamada das funções

In [152]:
node = tree.search(list_tasks)
node.value()

Tarefas no sistema


,task_id,task_name,status,created_at,due_date
0,1,Comprar leite,pendente,2023-10-01 09:00:00,2023-10-05
1,2,Enviar e-mail,concluída,2023-10-01 10:00:00,2023-10-02
2,3,Reunião com a equipe,pendente,2023-10-02 11:00:00,2023-10-04
3,4,Revisar relatório,pendente,2023-10-03 12:30:00,2023-10-05
4,5,Organizar arquivos,concluída,2023-10-04 14:15:00,2023-10-06


# Chatbot

In [180]:
def preprocess_prompt(prompt):
  prompt = prompt.lower()

  tokens = word_tokenize(prompt)
  tokens = [word for word in tokens if word not in stopwords.words('portuguese') and word not in string.punctuation]

  stemmer = RSLPStemmer()
  tokens = [stemmer.stem(word) for word in tokens]

  return tokens

In [178]:
def process_prompt(prompt) -> Callable:
    # Extrair tokens e remover stopword
    tokens = preprocess_prompt(prompt)
    print(tokens)

    entity = ""
    intention = ""

    res_function = tree.search(intention, entity)
    return res_function.value


In [177]:
def main():
  print("Olá, seja bem-vindo(a)!")
  # prompt = input("No que posso te ajudar hoje?")
  prompt="O que eu faço aaaa??"

  # Acessar resposta que é uma função
  funtion = process_prompt(prompt)
  # funtion()


In [ ]:
main()